In [31]:
from pyspark import SparkConf, SparkContext
conf = SparkConf().setMaster("local").setAppName("spark_sql_basic2")
sc = SparkContext(conf=conf)

In [3]:
# RDD만을 이용한 데이터 추출

In [35]:
movies_rdd = sc.parallelize([
    (1, ("어벤져스", "마블")),
    (2, ("슈퍼맨", "DC")),
    (3, ("배트맨", "DC")),
    (4, ("겨울왕국", "디즈니")),
    (5, ("아이언맨", "마블"))
])


attendances_rdd = sc.parallelize([
    (1, (13934592, "KR")),
    (2, (2182227,"KR")),
    (3, (4226242, "KR")),
    (4, (10303058, "KR")),
    (5, (4300365, "KR"))
])

In [36]:
# 마블 영화 중 관객 수가 500만 이상인 영화를 가져오기

In [37]:
# CASE1. join 먼저, filter 나중에
movie_att = movies_rdd.join(attendances_rdd)
movie_att.take(5)

[(2, (('슈퍼맨', 'DC'), (2182227, 'KR'))),
 (4, (('겨울왕국', '디즈니'), (10303058, 'KR'))),
 (1, (('어벤져스', '마블'), (13934592, 'KR'))),
 (3, (('배트맨', 'DC'), (4226242, 'KR'))),
 (5, (('아이언맨', '마블'), (4300365, 'KR')))]

In [38]:
movie_att.filter(
    lambda x : x[1][0][1] == "마블" and x[1][1][0] > 5000000
).collect()

[(1, (('어벤져스', '마블'), (13934592, 'KR')))]

In [39]:
# CASE 2. filter 먼저, join 나중에
filtered_movies = movies_rdd.filter(lambda x : x[1][1] == '마블')
filtered_att = attendances_rdd.filter(lambda x : x[1][0] > 5000000)

filtered_movies.join(filtered_att).collect()

[(1, (('어벤져스', '마블'), (13934592, 'KR')))]

In [40]:
sc.stop()

In [41]:
# Spark SQL 사용해 보기

In [42]:
from pyspark.sql import SparkSession # sql 쓰려면 sparksession이 필요
spark = SparkSession.builder.master("local").appName("spark-sql").getOrCreate()

In [43]:
# 컬럼 추가
movies = [
    (1, "어벤져스", "마블", 2012, 4, 26),
    (2, "슈퍼맨", "DC", 2013, 6, 13),
    (3, "배트맨", "DC", 2008, 8, 6),
    (4, "겨울왕국", "디즈니", 2014, 1, 16),
    (5, "아이언맨", "마블", 2008, 4, 30)
]

In [44]:
#스키마(헤더)를 알아야 한다.
movie_schema = ["id", "name", "company", "year", "month", "day"]

In [45]:
# 2. 데이터 프레임 만들기

In [46]:
df = spark.createDataFrame(data=movies, schema=movie_schema)

In [47]:
df.columns

['id', 'name', 'company', 'year', 'month', 'day']

In [48]:
df.dtypes

[('id', 'bigint'),
 ('name', 'string'),
 ('company', 'string'),
 ('year', 'bigint'),
 ('month', 'bigint'),
 ('day', 'bigint')]

In [49]:
df.show(3)

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  1|어벤져스|   마블|2012|    4| 26|
|  2|  슈퍼맨|     DC|2013|    6| 13|
|  3|  배트맨|     DC|2008|    8|  6|
+---+--------+-------+----+-----+---+
only showing top 3 rows



In [50]:
df.select("name").show()

+--------+
|    name|
+--------+
|어벤져스|
|  슈퍼맨|
|  배트맨|
|겨울왕국|
|아이언맨|
+--------+



In [51]:
df.filter(df.year >= 2010).show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  1|어벤져스|   마블|2012|    4| 26|
|  2|  슈퍼맨|     DC|2013|    6| 13|
|  4|겨울왕국| 디즈니|2014|    1| 16|
+---+--------+-------+----+-----+---+



In [52]:
df.select('day').show()

+---+
|day|
+---+
| 26|
| 13|
|  6|
| 16|
| 30|
+---+



In [53]:
df.select(['year','month','day']).show()  # 년, 월, 일

+----+-----+---+
|year|month|day|
+----+-----+---+
|2012|    4| 26|
|2013|    6| 13|
|2008|    8|  6|
|2014|    1| 16|
|2008|    4| 30|
+----+-----+---+



In [54]:
# 2013년 이후 영화
df.filter(df.year > 2013).show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  4|겨울왕국| 디즈니|2014|    1| 16|
+---+--------+-------+----+-----+---+



In [55]:
# 마블 영화, DC 같이 꺼내기
df.filter(df.company.isin(['마블', 'DC'])).show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  1|어벤져스|   마블|2012|    4| 26|
|  2|  슈퍼맨|     DC|2013|    6| 13|
|  3|  배트맨|     DC|2008|    8|  6|
|  5|아이언맨|   마블|2008|    4| 30|
+---+--------+-------+----+-----+---+



In [56]:
df.createOrReplaceTempView("movies") # 가상의 테이블 뷰 movies 생성

In [57]:
# 영화 이름만 가져오기

query = """

SELECT name
  FROM movies

"""
spark.sql(query).show()

+--------+
|    name|
+--------+
|어벤져스|
|  슈퍼맨|
|  배트맨|
|겨울왕국|
|아이언맨|
+--------+



In [67]:
# 2010년 이후에 개봉한 영화를 조회

query = """
SELECT name, year
    FROM movies
WHERE year > 2010

"""
spark.sql(query).show()

+--------+----+
|    name|year|
+--------+----+
|어벤져스|2012|
|  슈퍼맨|2013|
|겨울왕국|2014|
+--------+----+



In [68]:
# 2012년도 이전에 개봉한 영화의 이름과 회사를 출력
query = """

SELECT name, company
    FROM movies
WHERE year < 2012

"""
spark.sql(query).show()

+--------+-------+
|    name|company|
+--------+-------+
|  배트맨|     DC|
|아이언맨|   마블|
+--------+-------+



In [63]:
# like 문자열 데이터에서 특정 단어나 문장을 포함한 데이터를 찾을 때
# % 기호를 사용해서 문장이 매칭되는지 확인 가능!
# 제목이 ~~맨으로 끝나는 데이터의 모든 정보를 조회
query = """
select *
from movies
where name LIKE '%맨'

"""
spark.sql(query).show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  2|  슈퍼맨|     DC|2013|    6| 13|
|  3|  배트맨|     DC|2008|    8|  6|
|  5|아이언맨|   마블|2008|    4| 30|
+---+--------+-------+----+-----+---+



In [23]:
df

DataFrame[id: bigint, name: string, company: string, year: bigint, month: bigint, day: bigint]

In [64]:
# BETWEEN 특정 데이터와 데이터 사이를 조회
# 개봉 월이 4 ~ 8월 사이. 4 <= 개봉월 <= 8
query = """
select *
from movies
where month BETWEEN 4 AND 8
"""
spark.sql(query).show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  1|어벤져스|   마블|2012|    4| 26|
|  2|  슈퍼맨|     DC|2013|    6| 13|
|  3|  배트맨|     DC|2008|    8|  6|
|  5|아이언맨|   마블|2008|    4| 30|
+---+--------+-------+----+-----+---+



In [ ]:
# Join 구현하기

In [69]:
attendances = [
    (1, 13934592., "KR"),
    (2, 2182227.,"KR"),
    (3, 4226242., "KR"),
    (4, 10303058., "KR"),
    (5, 4300365., "KR")
]

In [70]:
# 직접 스키마 지정해 보기
from pyspark.sql.types import StringType, FloatType\
    , IntegerType\
    , StructType, StructField

In [71]:
# StructType 구조체 만들기
att_schema = StructType([ # 모든 컬럼의 타입을 통칭 - 컬럼 데이터의 집합
    StructField("id", IntegerType(), True), # StructField : 컬럼
    StructField("att", FloatType(), True),
    StructField("theater_country", StringType(), True)
])

In [72]:
att_df = spark.createDataFrame(
    data=attendances,
    schema=att_schema
)

att_df.dtypes

[('id', 'int'), ('att', 'float'), ('theater_country', 'string')]

In [73]:
att_df.createOrReplaceTempView("att")

In [74]:
att_df.select('*').show()

+---+-----------+---------------+
| id|        att|theater_country|
+---+-----------+---------------+
|  1|1.3934592E7|             KR|
|  2|  2182227.0|             KR|
|  3|  4226242.0|             KR|
|  4|1.0303058E7|             KR|
|  5|  4300365.0|             KR|
+---+-----------+---------------+



In [75]:
# df와 join
query = '''
select movies.id, movies.name, movies.company, att.att
from movies
join att ON movies.id = att.id
'''
spark.sql(query).show()

+---+--------+-------+-----------+
| id|    name|company|        att|
+---+--------+-------+-----------+
|  1|어벤져스|   마블|1.3934592E7|
|  2|  슈퍼맨|     DC|  2182227.0|
|  3|  배트맨|     DC|  4226242.0|
|  4|겨울왕국| 디즈니|1.0303058E7|
|  5|아이언맨|   마블|  4300365.0|
+---+--------+-------+-----------+



In [76]:
# SQL_최적화

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
    .appName("trip_count_sql")\
    .getOrCreate()

In [2]:
trip_file = "learning_spark_data/fhvhv_tripdata_2020-03.csv"

In [3]:
# inferSchema : 자동으로 스키마 예측하게 하기
data = spark.read.csv(trip_file, inferSchema=True, header=True)
data.show(3)

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0005|              B02510|2020-03-01 00:03:40|2020-03-01 00:23:39|          81|         159|   NULL|
|           HV0005|              B02510|2020-03-01 00:28:05|2020-03-01 00:38:57|         168|         119|   NULL|
|           HV0003|              B02764|2020-03-01 00:03:07|2020-03-01 00:15:04|         137|         209|      1|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
only showing top 3 rows



In [4]:
data.createOrReplaceTempView("mobility_data")

In [5]:
query = """
select *
from mobility_data
limit 5
"""
spark.sql(query).show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0005|              B02510|2020-03-01 00:03:40|2020-03-01 00:23:39|          81|         159|   NULL|
|           HV0005|              B02510|2020-03-01 00:28:05|2020-03-01 00:38:57|         168|         119|   NULL|
|           HV0003|              B02764|2020-03-01 00:03:07|2020-03-01 00:15:04|         137|         209|      1|
|           HV0003|              B02764|2020-03-01 00:18:42|2020-03-01 00:38:42|         209|          80|   NULL|
|           HV0003|              B02764|2020-03-01 00:44:24|2020-03-01 00:58:44|         256|         226|   NULL|
+-----------------+--------------------+-------------------+-------------------+

In [6]:
# 스파크 SQL을 사용하는 이유

In [7]:
query = """

select split(pickup_datetime, ' ')[0] as pickup_date, count(*) as trips
from mobility_data

group by pickup_date
"""

spark.sql(query).show()

+-----------+------+
|pickup_date| trips|
+-----------+------+
| 2020-03-03|697880|
| 2020-03-02|648986|
| 2020-03-01|784246|
| 2020-03-06|872012|
| 2020-03-05|731165|
| 2020-03-04|707879|
| 2020-03-09|628940|
| 2020-03-08|731222|
| 2020-03-07|886071|
| 2020-03-10|626474|
| 2020-03-12|643257|
| 2020-03-11|628601|
| 2020-03-16|391518|
| 2020-03-13|660914|
| 2020-03-15|448125|
| 2020-03-14|569397|
| 2020-03-26|141607|
| 2020-03-25|141088|
| 2020-03-20|261900|
| 2020-03-24|141686|
+-----------+------+
only showing top 20 rows



In [8]:
# 실행 계획 살펴보기
spark.sql(query).explain(True)

== Parsed Logical Plan ==
'Aggregate ['pickup_date], ['split('pickup_datetime,  )[0] AS pickup_date#129, 'count(1) AS trips#130]
+- 'UnresolvedRelation [mobility_data], [], false

== Analyzed Logical Plan ==
pickup_date: string, trips: bigint
Aggregate [split(cast(pickup_datetime#19 as string),  , -1)[0]], [split(cast(pickup_datetime#19 as string),  , -1)[0] AS pickup_date#129, count(1) AS trips#130L]
+- SubqueryAlias mobility_data
   +- View (`mobility_data`, [hvfhs_license_num#17,dispatching_base_num#18,pickup_datetime#19,dropoff_datetime#20,PULocationID#21,DOLocationID#22,SR_Flag#23])
      +- Relation [hvfhs_license_num#17,dispatching_base_num#18,pickup_datetime#19,dropoff_datetime#20,PULocationID#21,DOLocationID#22,SR_Flag#23] csv

== Optimized Logical Plan ==
Aggregate [_groupingexpression#134], [_groupingexpression#134 AS pickup_date#129, count(1) AS trips#130L]
+- Project [split(cast(pickup_datetime#19 as string),  , -1)[0] AS _groupingexpression#134]
   +- Relation [hvfhs_lice

In [9]:
# 두번째 쿼리
spark.sql("""select 
                pickup_date, 
                count(*) as trips
             from ( select
                          split(pickup_datetime, ' ')[0] as pickup_date
                          from mobility_data )
             group by pickup_date""").explain(True)

== Parsed Logical Plan ==
'Aggregate ['pickup_date], ['pickup_date, 'count(1) AS trips#138]
+- 'SubqueryAlias __auto_generated_subquery_name
   +- 'Project ['split('pickup_datetime,  )[0] AS pickup_date#137]
      +- 'UnresolvedRelation [mobility_data], [], false

== Analyzed Logical Plan ==
pickup_date: string, trips: bigint
Aggregate [pickup_date#137], [pickup_date#137, count(1) AS trips#138L]
+- SubqueryAlias __auto_generated_subquery_name
   +- Project [split(cast(pickup_datetime#19 as string),  , -1)[0] AS pickup_date#137]
      +- SubqueryAlias mobility_data
         +- View (`mobility_data`, [hvfhs_license_num#17,dispatching_base_num#18,pickup_datetime#19,dropoff_datetime#20,PULocationID#21,DOLocationID#22,SR_Flag#23])
            +- Relation [hvfhs_license_num#17,dispatching_base_num#18,pickup_datetime#19,dropoff_datetime#20,PULocationID#21,DOLocationID#22,SR_Flag#23] csv

== Optimized Logical Plan ==
Aggregate [pickup_date#137], [pickup_date#137, count(1) AS trips#138L]
+- Pro

In [10]:
spark.stop()

In [12]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("trip_count_sql").getOrCreate()

In [ ]:
trip_file = "fhvhv_tripdata_2020-03.csv"
zone_file = "taxi+_zone_lookup.csv"

In [16]:
# 운행 데이터 프레임 생성, Zone 데이터 프레임 생성
trip_data = spark.read.format("csv")\
    .option('header','true')\
    .option('inferSchema', 'true')\
    .load('learning_spark_data/fhvhv_tripdata_2020-03.csv')

zone_data = spark.read.format("csv")\
    .option('header','true')\
    .option('inferSchema','true')\
    .load('learning_spark_data/taxi+_zone_lookup.csv')

In [23]:
trip_data.printSchema()
# 각 운행에 대한 상세 정보
# 운송사업자 ID, 배차 지점 ID, 승차 시각, 하차 시각, 승차 지역 ID, 하차 지역 ID, 공유 운행 여부

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: integer (nullable = true)



In [24]:
zone_data.printSchema()
# locationid에 대한 지리적 정보
# 고유 지역, 자치구 이름, 세부적인 지역 이름, 서비스 구역

root
 |-- LocationID: integer (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)



In [19]:
zone_data.show(3)

+----------+-------+--------------------+------------+
|LocationID|Borough|                Zone|service_zone|
+----------+-------+--------------------+------------+
|         1|    EWR|      Newark Airport|         EWR|
|         2| Queens|         Jamaica Bay|   Boro Zone|
|         3|  Bronx|Allerton/Pelham G...|   Boro Zone|
+----------+-------+--------------------+------------+
only showing top 3 rows



In [22]:
trip_data.show(3)

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0005|              B02510|2020-03-01 00:03:40|2020-03-01 00:23:39|          81|         159|   NULL|
|           HV0005|              B02510|2020-03-01 00:28:05|2020-03-01 00:38:57|         168|         119|   NULL|
|           HV0003|              B02764|2020-03-01 00:03:07|2020-03-01 00:15:04|         137|         209|      1|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
only showing top 3 rows



In [ ]:
## 승차 Location(PULocationID)별 개수 세기
# 하차 Location(DOLocationID)별 개수 세기
#HV0003 운송사업자의 승차 지역별 트립 건수를 집계하고, 
#가장 많은 운송사업자순으로 정렬하는 분석 쿼리  hvfhs_license_num
#운송사별 운행 건수 비교
#승차 위치 Borough별 운행 건수
#서비스 존별 승차/하차 건수

In [26]:
## 승차 Location(PULocationID)별 개수 세기
trip_data.groupBy('PULocationID').count().show()

+------------+------+
|PULocationID| count|
+------------+------+
|         148|116205|
|         243| 87431|
|          31|  5285|
|         137| 85552|
|          85| 46120|
|         251|  9080|
|          65| 66622|
|         255|113947|
|          53| 17571|
|         133| 27200|
|          78| 76155|
|         108| 20378|
|         155| 39527|
|         211| 61075|
|         193| 20111|
|          34| 11823|
|         115| 10806|
|         126| 52833|
|         101|  8983|
|          81| 41425|
+------------+------+
only showing top 20 rows



In [27]:
# 하차 Location(DOLocationID)별 개수 세기
trip_data.groupBy('DOLocationID').count().show()

+------------+------+
|DOLocationID| count|
+------------+------+
|         148| 91601|
|         243| 86795|
|          31|  5526|
|          85| 44509|
|         137| 80098|
|         251|  8525|
|          65| 58888|
|         255|105051|
|          53| 19013|
|         133| 27760|
|          78| 74447|
|         155| 42239|
|         108| 21354|
|         211| 54176|
|         193| 19104|
|          34| 12392|
|         115|  9809|
|         101|  7218|
|         126| 59027|
|          81| 38445|
+------------+------+
only showing top 20 rows



In [36]:
#HV0003 운송사업자의 승차 지역별 트립 건수를 집계
hv0003_trips = trip_data.filter(trip_data.hvfhs_license_num == "HV0003") # hvfhs_license_num = HB0003 필터링
hv0003_pu_count = hv0003_trips.groupBy("PULocationID").count() # pulocaionid 별 trip count
hv0003_pu_with_zone = hv0003_pu_count.join(
    zone_data,
    hv0003_pu_count.PULocationID == zone_data.LocationID,
    how="left"
).select("PULocationID", "count","Borough", "Zone")

In [37]:
hv0003_pu_with_zone.orderBy("count",ascending=False).show()

+------------+------+---------+--------------------+
|PULocationID| count|  Borough|                Zone|
+------------+------+---------+--------------------+
|          61|163091| Brooklyn| Crown Heights North|
|          76|134198| Brooklyn|       East New York|
|         132|114179|   Queens|         JFK Airport|
|          79|112017|Manhattan|        East Village|
|          37|110150| Brooklyn|      Bushwick South|
|          42|108070|Manhattan|Central Harlem North|
|         138|104119|   Queens|   LaGuardia Airport|
|         244| 97324|Manhattan|Washington Height...|
|          89| 95724| Brooklyn|Flatbush/Ditmas Park|
|          39| 94484| Brooklyn|            Canarsie|
|         231| 94155|Manhattan|TriBeCa/Civic Center|
|           7| 92676|   Queens|             Astoria|
|          17| 90352| Brooklyn|             Bedford|
|         161| 90261|Manhattan|      Midtown Center|
|         225| 88749| Brooklyn|  Stuyvesant Heights|
|         234| 88372|Manhattan|            Uni

In [38]:
#가장 많은 운송사업자순으로 정렬하는 분석 쿼리  hvfhs_license_num
trip_data.groupBy('hvfhs_license_num').count().orderBy('count',ascending=False).show()

+-----------------+-------+
|hvfhs_license_num|  count|
+-----------------+-------+
|           HV0003|9836763|
|           HV0005|3219535|
|           HV0004| 336606|
+-----------------+-------+



In [42]:
#운송사별 운행 건수 비교
trip_data.groupBy('hvfhs_license_num').count().show()

+-----------------+-------+
|hvfhs_license_num|  count|
+-----------------+-------+
|           HV0004| 336606|
|           HV0005|3219535|
|           HV0003|9836763|
+-----------------+-------+



In [48]:
#승차 위치 Borough별 운행 건수
trip_with_zone = trip_data.join(
    zone_data,
    trip_data.PULocationID == zone_data.LocationID,
    how='left'
)

In [49]:
borough_trip_counts = trip_with_zone.groupBy('Borough').count().orderBy('count',ascending=False)
borough_trip_counts.show()

+-------------+-------+
|      Borough|  count|
+-------------+-------+
|    Manhattan|4953140|
|     Brooklyn|3735764|
|       Queens|2437383|
|        Bronx|2086592|
|Staten Island| 178818|
|      Unknown|    845|
|          EWR|    362|
+-------------+-------+



In [59]:
#서비스 존별 승차/하차 건수

# 서비스 존별 승차 건수
from pyspark.sql.functions import col
# 승차 위치 기준 조인
pickup_join = trip_data.join(
    zone_data,
    trip_data.PULocationID == zone_data.LocationID,
    how='left'
)

# 서비스 존별 승차 건수 집계
pickup_service_zone = pickup_join.groupBy('service_zone') \
    .count() \
    .withColumnRenamed('count','pickup_count') \
    .orderBy(col('pickup_count').desc())

pickup_service_zone.show()

+------------+------------+
|service_zone|pickup_count|
+------------+------------+
|   Boro Zone|     9046897|
| Yellow Zone|     4025190|
|    Airports|      319610|
|         N/A|         845|
|         EWR|         362|
+------------+------------+



In [62]:
# 서비스 존별 하차 건수
# 하차 위치 기준 조인
dropff_join = trip_data.join(
    zone_data,
    trip_data.DOLocationID == zone_data.LocationID,
    how='left'
)

# 서비스 존별 하차 건수 집계 
dropoff_service_zone = dropff_join.groupBy('service_zone')\
    .count()\
    .withColumnRenamed('count','dropoff_count')\
    .orderBy(col('dropoff_count').desc())

dropoff_service_zone.show()

+------------+-------------+
|service_zone|dropoff_count|
+------------+-------------+
|   Boro Zone|      8885136|
| Yellow Zone|      3643787|
|    Airports|       411156|
|         N/A|       387759|
|         EWR|        65066|
+------------+-------------+

